In [0]:
#File path
dbutils.fs.ls('/Volumes/workspace/stream/strm-vol/jsonsource/')


##Read json data (Batch)

In [0]:
"""
df = spark.read.format('json') \
    .option('inferSchema', True) \
        .option('multiline', True) \
            .load('/Volumes/workspace/stream/strm-vol/jsonsource')

"""



In [0]:

"""
from pyspark.sql.functions import *
df = df.withColumn("items",explode_outer('items')) \
    .withColumn('metadata', explode_outer('metadata'))

df.display()

"""



In [0]:
"""
df = df.select("order_id", "timestamp", "customer.customer_id","customer.name","customer.email","customer.address.city","customer.address.postal_code","customer.address.country","items.item_id", "items.price","items.product_name","items.quantity",'payment.method',"payment.transaction_id",'metadata.key','metadata.value')

df.select("*").display()
"""

In [0]:
"""
df.printSchema()

"""

## Read Streaming Data

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

my_schema = """order_id STRING,
        timestamp STRING,
        customer STRUCT<
          customer_id: INT,
          name: STRING,
          email: STRING,
          address: STRUCT<
            city: STRING,
            postal_code: STRING,
            country: STRING
          >
        >,
        items ARRAY<STRUCT<
          item_id: STRING,
          product_name: STRING,
          quantity: INT,
          price: DOUBLE
        >>,
        payment STRUCT<
          method: STRING,
          transaction_id: STRING
        >,
        metadata ARRAY<STRUCT<
          key: STRING,
          value: STRING
        >>"""

In [0]:
# Schema inference
# spark.conf.set("spark.sql.streaming.schemaInference", "true")

df = spark.readStream.format("json") \
  .option("multiline", "true") \
    .schema(my_schema) \
      .load("/Volumes/workspace/stream/strm-vol/jsonsource")


# transformation code

df = df.withColumn("items",explode_outer('items')) \
    .withColumn('metadata', explode_outer('metadata'))

df = df.select("order_id", "timestamp", "customer.customer_id","customer.name","customer.email","customer.address.city","customer.address.postal_code","customer.address.country","items.item_id", "items.price","items.product_name","items.quantity",'payment.method',"payment.transaction_id",'metadata.key','metadata.value')


In [0]:

df.writeStream.format("delta")\
    .outputMode("append")\
        .trigger(once=True)\
            .option("path","/Volumes/workspace/stream/strm-vol/jsonsink/data")\
                .option("checkpointLocation", "/Volumes/workspace/stream/strm-vol/jsonsink/checkpoint")\
                    .start()


In [0]:
%sql
select * from delta.`/Volumes/workspace/stream/strm-vol/jsonsink/data`